In [1]:
# HIDE
try:
    import setup
except:
    pass
from notebook_helper import *
notebook_setup()

from IPython.display import clear_output
from ipywidgets import interactive
from modules import la

# Council emissions profile demo

This notebook is a demo of v2 of the emissions profile and distance tool. This may take a few seconds to load.

In [2]:
def dmd(x): return display(md(x))


options = {"Overall": "composite",
           "Emissions": "emissions",
           "Nearby councils": "geographic_distance",
           "Similar deprivation profile": "imd",
           "Similar rural/urbal profile": "ruc",
           }


dmd("Loading...")

la_df = pd.read_csv(Path("data", "la_details.csv")).set_index("local-authority-code")

name_to_code = la_df.reset_index().common.to_map(
    "official-name", "local-authority-code")


class ComparisonDisplay:

    def __init__(self, comparison_type: str):

        folder = Path("data", "outputs", comparison_type)

        self.label_desc = pd.read_csv(folder / "label_desc.csv")
        self.labels = pd.read_csv(folder / "la_labels.csv")
        self.labels = self.labels.merge(
            self.label_desc).set_index("local-authority-code")
        self.ldf = la_df.join(self.labels)
        self.distance = (pd.read_csv(folder / "distance_map.csv")
                         .loc[lambda x: x["position"] < 11])

    def matches_for_council(self, council_slug: str):
        def mask(x): return x["local-authority-code_A"] == council_slug
        df = self.distance.loc[mask]
        df = (df.drop(columns=["local-authority-code_A"])
              .set_index("local-authority-code_B")
              .join(self.ldf[["official-name", "label"]]))

        df = df.reset_index()[["official-name", "match", "label"]]

        return df

def display_council(comparison_type: str, council_name: str):

    comparison_slug = options[comparison_type]
    council_slug = name_to_code[council_name]

    c = ComparisonDisplay(comparison_slug)

    details = c.ldf.loc[council_slug]
    dmd(f'## {details["official-name"]}')
    dmd(f'**Type**: {details["local-authority-type-name"]}')
    dmd(f'**Cluster**: {details["label"]}')
    dmd(f'**Cluster description**: {details["desc"]}')

    dmd("### Nearest neighbours")
    df = c.matches_for_council(council_slug)
    display(df)
clear_output()
interactive(display_council, comparison_type=options.keys(),
            council_name=la_df["official-name"].sort_values().tolist())


interactive(children=(Dropdown(description='comparison_type', options=('Overall', 'Emissions', 'Nearby council…